In [ ]:
!pip install zenML

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 795 kB 27.6 MB/s 
     |████████████████████████████████| 181 kB 39.5 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |████████████████████████████████| 561 kB 44.5 MB/s 
     |████████████████████████████████| 1.3 MB 47.5 MB/s 
     |████████████████████████████████| 636 kB 62.7 MB/s 
     |████████████████████████████████| 235 kB 58.8 MB/s 
     |████████████████████████████████| 10.9 MB 45.0 MB/s 
     |████████████████████████████████| 11.1 MB 55.8 MB/s 
     |████████████████████████████████| 96 kB 6.3 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 147 kB 53.6 MB/s 
     |████████████████████████████████| 40 kB 6.2 MB/s 
     |████████████████████████████████| 6.6 MB 52.9 MB/s 
     |████████████████████████████████| 135 kB 56.7 MB/s 
     |███████████████████████████████

# New Section

In [ ]:
!zenml init

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.11.0
Creating default profile...
Initializing profile default...
Registering default stack...
Registered stack component with type 'orchestrator' and name 'default'.
Registered stack component with type 'metadata_store' and name 'default'.
Registered stack component with type 'artifact_store' and name 'default'.
Registered stack with name 'default'.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
Created and activated default profile.
ZenML repository initialized at /content.
⠸ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.

The local active profile was initialized to 'default' and the local active stack
to 'default'. This local configuration will only take effect when you're running
ZenML from the initialize

In [ ]:
!zenml integration install sklearn -y # code is not workig with out integrating with some libraires|

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
⠏ Installing integrations...


In [ ]:
from zenml.pipelines import pipeline
from zenml.steps import step, Output


INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
from zenml.materializers.base_materializer import BaseMaterializer
from zenml.artifacts import DataArtifact

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 35.4 MB/s 
     |████████████████████████████████| 6.6 MB 54.6 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 


In [ ]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [ ]:
!pip3 install -q git+https://github.com/vasudevgupta7/gsoc-wav2vec2@main
!sudo apt-get install -y libsndfile1-dev
!pip3 install -q SoundFile  

     |████████████████████████████████| 1.8 MB 33.0 MB/s 
     |████████████████████████████████| 50 kB 6.7 MB/s 
     |████████████████████████████████| 153 kB 48.6 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1-dev is already the newest version (1.0.28-4ubuntu0.18.04.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from wav2vec2 import Wav2Vec2Config
from wav2vec2 import CTCLoss
import soundfile as sf

In [ ]:
@step
def upload_audio_file() -> Output(path= str):
  path="/content/amanda_gorman.flac"
  return path


In [ ]:
@step
def asr_transcript(audio_file: str) -> Output(transcript= str):

      tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
      model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
      transcript = ""

      # Stream over 20 seconds chunks
      stream = librosa.stream(
          audio_file, block_length=20, frame_length=16000, hop_length=16000
      )

      for speech in stream:
          if len(speech.shape) > 1:
              speech = speech[:, 0] + speech[:, 1]

          input_values = tokenizer(speech, return_tensors="pt").input_values
          logits = model(input_values).logits

          predicted_ids = torch.argmax(logits, dim=-1)
          transcription = tokenizer.decode(predicted_ids[0])
          transcript += transcription.lower() + " "
      print(transcript)     
          
      return transcript

In [ ]:
# asr_transcript.entrypoint(audio_file="/content/Wave_files_demos_Welcome (5).wav")

In [ ]:
@pipeline
def model_1(step_1,step_2):
    """Links all the steps together in a pipeline"""
    path=step_1()
    transcription=step_2(path)

In [ ]:
model_1(
    step_1=upload_audio_file(),
    step_2=asr_transcript(),
).run(run_name="run_2")

Creating run for pipeline: model_1
Cache enabled for pipeline model_1
Using stack default to run pipeline model_1...
Step upload_audio_file has started.
Using cached version of upload_audio_file.
Step upload_audio_file has finished in 0.062s.
Step asr_transcript has started.


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# "@pipeline"
# def model_2(step_1,step_2,step_3,step_4):
#     """Links all the steps together in a pipeline"""
#     path=step_1()
#     model=step_2().with_return_materializers(MyMaterializer_model)
#     token=step_3().with_return_materializers(MyMaterializer_token)
#     transcription=step_4(path,model,token)
    


In [ ]:
# class MyMaterializer_token(BaseMaterializer):
#     ASSOCIATED_TYPES = (Wav2Vec2Tokenizer,)
#     ASSOCIATED_ARTIFACT_TYPES = (DataArtifact,)
#     def handle_input() -> Wav2Vec2ForCTC:
#         """Read from artifact store"""
#         tokenizer_transcribe = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
#         return tokenizer_transcribe

In [ ]:
# class MyMaterializer_model(BaseMaterializer):
#     ASSOCIATED_TYPES = (Wav2Vec2ForCTC,)
#     ASSOCIATED_ARTIFACT_TYPES = (DataArtifact,)
#     def handle_input() -> Wav2Vec2ForCTC:
#         """Read from artifact store"""
#         model_transcribe = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
#         return model_transcribe


In [ ]:
# model_2(
#     step_1=upload_audio_file(),
#     step_2=model_load().with_return_materializers(MyMaterializer_model),
#     step_3=token_load().with_return_materializers(MyMaterializer_token),
#     step_4=asr_transcript(),
# ).run(run_name="run_3")